In [ ]:
import json
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Input,Conv1D,MaxPool1D,LSTM,Flatten,Dropout,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!wget https://github.com/Yossioren/pp0/blob/main/occupancy_small.npz
!wget https://github.com/Yossioren/pp0/blob/main/sweeps_small.npz
!wget https://github.com/Yossioren/pp0/blob/main/dns_small.npz
!wget https://github.com/Yossioren/pp0/blob/main/sns_small.npz

In [ ]:
def cnn_lstm(input_vector, output_size=20, filters=256, strides=3, pool_size=4, units=32, dropout=0.7, lr=0.001)->tf.keras.Model:
    inp = Input((input_vector, 1))
    x = Conv1D( filters, kernel_size=16, strides=strides, activation='relu')(inp)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    x = Conv1D(filters, kernel_size=8, strides=strides, activation='relu', padding='same')(x)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    x = LSTM(units, return_sequences=True, recurrent_activation='hard_sigmoid')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    preds = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=inp, outputs=preds)
    opt = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])  # optimizer, metrics
    return model

In [ ]:
def small_cnn_lstm(input_vector, output_size=20, filters=256, strides=3, pool_size=4, units=32, dropout=0.7, kernel_size=32, lr=0.001):
    inp = Input((input_vector, 1))
    x = Conv1D(filters, kernel_size=kernel_size, strides=strides, activation='relu')(inp)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    x = LSTM(units, return_sequences=True, recurrent_activation='hard_sigmoid')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    preds = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=inp, outputs=preds)
    opt = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])  # optimizer, metrics
    return model


In [ ]:
def classify(dataset,n_validation=1):
    X=np.array( dataset['X'],dtype=np.float32)
    X=X.reshape(*X.shape,-1)
    y=np.array( dataset['y'])

    lab_enc=LabelEncoder()
    lab_hot=OneHotEncoder()
    y_num= lab_enc.fit_transform(y)
    y_hot=lab_hot.fit_transform(y_num.reshape(-1,1))
    y_hot=y_hot.toarray()

    skf = StratifiedShuffleSplit(n_splits=10,random_state=666)
    preds=[]
    true=[]
    counter=0
    for train_index, test_index in skf.split(X, y_num):
        if counter>=n_validation:
            break
        else:
            counter+=1
        X_train, X_test = X[train_index], X[test_index]
        mean=X_train.mean()
        std=X_train.std()
        X_train_norm=(X_train-mean+1e-10)/std
        X_test_norm=(X_test-mean+1e-10)/std

        y_train, y_test = y_hot[train_index], y_num[test_index]
        length=X_train.shape[1]
        if length<1000:
            model=small_cnn_lstm(X_train.shape[1])
        else:
            model=cnn_lstm(X_train.shape[1])
        model.fit(X_train_norm,y_train,validation_split=0.1,epochs=100,verbose=2,callbacks=[EarlyStopping( patience=2,restore_best_weights=True)])
        preds.extend( model.predict(X_test_norm).argmax(axis=-1))
        true.extend(y_test)
        
    print(classification_report(true, preds))
    cm = confusion_matrix(true,preds)
    fig,ax=plt.subplots(figsize=(7,7))
    sns.heatmap(cm,ax=ax,cmap='RdYlGn')
    plt.show()



    

In [ ]:
cache_occupancy_data='default_small.npz'#slow
sweeps_data='sweeps_small.npz'#fast
sns_data='sns_small.npz'
dns_data='dns_small.npz'

In [ ]:
path=sweeps_data

In [ ]:
data=np.load(path)
classify(data)